# reaction-network (Demo Notebook): Enumerators

### Author: Matthew McDermott
Last Updated: 12/07/21

**If you use this code in your work, please consider citing the following paper:**

McDermott, M. J., Dwaraknath, S. S., and Persson, K. A. (2021). A graph-based network for predicting chemical reaction pathways in solid-state materials synthesis. 
Nature Communications, 12(1). https://doi.org/10.1038/s41467-021-23339-x

### Imports

In [1]:
import logging
from pprint import pprint

from pymatgen.core.composition import Composition, Element
from pymatgen.ext.matproj import MPRester

from rxn_network.enumerators.basic import BasicEnumerator, BasicOpenEnumerator
from rxn_network.enumerators.minimize import MinimizeGibbsEnumerator, MinimizeGrandPotentialEnumerator
from rxn_network.fireworks import EnumeratorFW
from rxn_network.costs.softplus import Softplus
from rxn_network.entries.entry_set import GibbsEntrySet

%load_ext autoreload
%autoreload 2

## Downloading and modifying entries

First, we acquire entries for phases in the Y-Mn-O chemical system from the Materials Project (MP), a computed materials database containing calculations for over 130,000 materials.

In [59]:
with MPRester() as mpr:  # insert your Materials Project API key here if it's not stored in .pmgrc.yaml
    entries = mpr.get_entries_in_chemsys("Y-Mn-O", inc_structure="final")

The `GibbsEntrySet` class allows us to automatically convered `ComputedStructureEntry` objects downloaded from the MP database into `GibbsComputedEntry` objects, where DFT-calculated energies have been converted to an AI-estimated equivalent values of the Gibbs free energies of formation, $\Delta G_f$ for all entries at the specified temperature.

For more information, check out the citation in the documentation for `GibbsComputedEntry`.

In [64]:
temp = 900  # units: Kelvin
gibbs_entries = GibbsEntrySet.from_entries(entries, temp)

[autoreload of rxn_network.entries.experimental failed: Traceback (most recent call last):
  File "/Users/mcdermott/opt/miniconda3/envs/rn_dev/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/mcdermott/opt/miniconda3/envs/rn_dev/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/mcdermott/opt/miniconda3/envs/rn_dev/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/Users/mcdermott/opt/miniconda3/envs/rn_dev/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "/Users/mcdermott/opt/miniconda3/envs/rn_dev/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/Users/mcdermott/opt/miniconda3/en

We can print the entries by calling `.entries` or `.entries_list`:

In [61]:
gibbs_entries.entries

{GibbsComputedEntry | mp-1062072 | Y1 Mn2 (YMn2)
 Gibbs Energy (800 K) = 2.2154,
 GibbsComputedEntry | mp-1086672 | Mn2 O6 (MnO3)
 Gibbs Energy (800 K) = -2.6940,
 GibbsComputedEntry | mp-1105767 | Mn6 O12 (MnO2)
 Gibbs Energy (800 K) = -23.9704,
 GibbsComputedEntry | mp-1172875 | Mn32 O48 (Mn2O3)
 Gibbs Energy (800 K) = -125.2360,
 GibbsComputedEntry | mp-1178684 | Y2 O4 (YO2)
 Gibbs Energy (800 K) = -15.4520,
 GibbsComputedEntry | mp-1180876 | Mn12 O16 (Mn3O4)
 Gibbs Energy (800 K) = -41.8356,
 GibbsComputedEntry | mp-1187739 | Y3 (Y)
 Gibbs Energy (800 K) = 0.0000,
 GibbsComputedEntry | mp-1187855 | Y2 O2 (YO)
 Gibbs Energy (800 K) = -10.8508,
 GibbsComputedEntry | mp-1189335 | Y4 O12 (YO3)
 Gibbs Energy (800 K) = -23.7903,
 GibbsComputedEntry | mp-1189857 | Mn6 O12 (MnO2)
 Gibbs Energy (800 K) = -24.3435,
 GibbsComputedEntry | mp-1203190 | Mn32 O48 (Mn2O3)
 Gibbs Energy (800 K) = -78.7365,
 GibbsComputedEntry | mp-1204718 | Mn32 O48 (Mn2O3)
 Gibbs Energy (800 K) = -117.8349,
 Gibbs

The `GibbsEntrySet` class has many helpful functions, such as the following `filter_by_stability()` function, which automatically removes entries which are a specified energy per atom above the convex hull of stability:

In [5]:
filtered_entries = gibbs_entries.filter_by_stability(0.025)

You should now see a much shorter list of entries within the Y-Mn-O system (< 25 meV/atom below the hull)

In [6]:
filtered_entries.entries_list

[GibbsComputedEntry | mp-35 | Mn29 (Mn)
 Gibbs Energy (800 K) = 0.0000,
 GibbsComputedEntry | mp-1172875 | Mn32 O48 (Mn2O3)
 Gibbs Energy (800 K) = -125.2360,
 GibbsComputedEntry | mp-18759 | Mn6 O8 (Mn3O4)
 Gibbs Energy (800 K) = -22.7493,
 GibbsComputedEntry | mp-18922 | Mn5 O8 (Mn5O8)
 Gibbs Energy (800 K) = -19.4691,
 GibbsComputedEntry | mp-999539 | Mn4 O4 (MnO)
 Gibbs Energy (800 K) = -12.8711,
 GibbsComputedEntry | mp-1279979 | Mn1 O2 (MnO2)
 Gibbs Energy (800 K) = -4.0968,
 GibbsComputedEntry | mp-12957 | O8 (O2)
 Gibbs Energy (800 K) = 0.0000,
 GibbsComputedEntry | mp-1187739 | Y3 (Y)
 Gibbs Energy (800 K) = 0.0000,
 GibbsComputedEntry | mp-18831 | Y4 Mn4 O14 (Y2Mn2O7)
 Gibbs Energy (800 K) = -53.2849,
 GibbsComputedEntry | mp-2652 | Y16 O24 (Y2O3)
 Gibbs Energy (800 K) = -141.2242,
 GibbsComputedEntry | mp-22508 | Y1 Mn12 (YMn12)
 Gibbs Energy (800 K) = -0.1268,
 GibbsComputedEntry | mp-510598 | Y4 Mn8 O20 (YMn2O5)
 Gibbs Energy (800 K) = -70.2802,
 GibbsComputedEntry | mp-19

## Running enumerators

There are several types of enumerator classes contained within `rxn_network.enumerators`: These are:

1. `BasicEnumerator`: use a combinatorial approach to identify all possible (closed) reactions within a set of entries
2. `BasicOpenEnumerator`: use a combinatorial approach to identify all **open** reactions within a set of entries and a list of specified open entries/elements
3. `MinimizeGibbsEnumerator`: use a thermodynamic approach to identify all reactions within a set of entries that are predicted by minimizing Gibbs free energy between a set of two reacting phases touching at an interface
4. `MinimizeGrandPotentialEnumerator`: use a thermodynamic approach to identify all reactions within a set of entries that are predicted by minimizing the grand potential between a set of two reacting phases touching at an interface with an **open** element at a specified chemical potential

### Basic enumerators
We first create a basic enumerator object by initializing one from the `BasicEnumerator` class:

In [7]:
be = BasicEnumerator()

The `BasicEnumerator` class, as is true for all other enumerator classes, can be provided with several arguments for customizing the enumerator output:

- **precursors**: Optional list of precursor formulas; only reactions which contain at least these phases as reactants will be enumerated.
- **target**: Optional formula of target; only reactions which include formation of this target will be enumerated.
- **calculators**: Optional list of Calculator object names; see calculators module for options (e.g., ["ChempotDistanceCalculator])
- **n**: Maximum reactant/product cardinality; i.e., largest possible number of entries on either side of the reaction. Defaults to 2.
- **exclusive_precursors**: Whether to consider only reactions that have reactants which are a subset of the provided list of precursors. Defaults to True.
- **exclusive_targets**: Whether to consider only reactions that make the provided target directly (i.e. with no byproducts). Defualts to False.
- **remove_unbalanced**: Whether to remove reactions which are unbalanced. Defaults to True.
- **remove_changed**: Whether to remove reactions which can only be balanced by removing a reactant/product or having it change sides. Defaults to True.

Note that the default arguments are good for generating a list of simple (unconstrained) reactions, as we might build for a reaction network. Run the following cell:

In [8]:
all_rxns = be.enumerate(filtered_entries)

BasicEnumerator:   0%|          | 0/4 [00:00<?, ?it/s]

This should complete somewhat quickly (within a few seconds). As a result, a list of ~800 generated reactions will be stored within the `all_rxns` object:

In [9]:
print(len(all_rxns))

800


In [10]:
pprint(all_rxns[0:10])  # first 10 reactions

[YMnO3 + 13 Mn -> YMn12 + Mn2O3,
 YMnO3 + 0.5 MnO2 -> 0.5 Y2Mn2O7 + 0.5 MnO,
 Mn + 0.03865 Y2O3 -> 0.01449 Mn5O8 + 0.07729 YMn12,
 Y2Mn2O7 + 26.67 Mn -> 2 YMn12 + 2.333 Mn2O3,
 0.5 Mn2O3 -> Mn + 0.75 O2,
 0.6667 Mn5O8 + Y2O3 -> Y2Mn2O7 + 1.333 MnO,
 0.6667 Y2Mn2O7 + 0.3333 Mn -> YMnO3 + 0.3333 YMn2O5,
 MnO + YMn2O5 -> YMnO3 + Mn2O3,
 0.02941 Y2Mn2O7 + 0.07843 YMn12 -> Mn + 0.06863 Y2O3,
 MnO + 0.02083 Y -> 0.25 Mn3O4 + 0.02083 YMn12]


Looking at the list of reactions above, we see that all reactions are stoichiometrically balanced. If we look at any particular reaction object, we find that the reaction energy and uncertainty can automatically be calculated using the entry energies:

In [11]:
r = all_rxns[0]
print(r)
print(r.energy_per_atom, "±", r.energy_uncertainty_per_atom, "eV/atom")

YMnO3 + 13 Mn -> YMn12 + Mn2O3
0.27845742189028416 ± 0.05471587667664505 eV/atom


If we want to generate only reactions which contain specific precursors, we can supply them when we initialize the `BasicEnumerator` object.

In [12]:
be_precursors = BasicEnumerator(precursors=["Y2O3"])
y2o3_rxns_exclusive = be_precursors.enumerate(filtered_entries)

BasicEnumerator:   0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
y2o3_rxns_exclusive

[0.5 Y2O3 -> Y + 0.75 O2]

Note that by default, this only produces reactions which exclusively have the provided precursor(s). To include reactions that contain this precursor (non-exclusively), set the `exclusive_precursors=False`:

In [14]:
be_precursors = BasicEnumerator(precursors=["Y2O3"], exclusive_precursors=False)
y2o3_rxns = be_precursors.enumerate(filtered_entries)

BasicEnumerator:   0%|          | 0/4 [00:00<?, ?it/s]

In [15]:
y2o3_rxns

[Mn5O8 + 0.2083 Y2O3 -> 0.4167 YMn12 + 4.312 O2,
 Mn + 0.03922 Y2O3 -> 0.05882 MnO2 + 0.07843 YMn12,
 Mn + 0.03865 Y2O3 -> 0.01449 Mn5O8 + 0.07729 YMn12,
 0.3333 Mn3O4 + 0.2222 Y2O3 -> MnO2 + 0.4444 Y,
 0.6667 Mn5O8 + Y2O3 -> Y2Mn2O7 + 1.333 MnO,
 1.667 Mn3O4 + 0.4444 Y2O3 -> Mn5O8 + 0.8889 Y,
 0.5 Y2O3 + 2.5 Mn2O3 -> Mn3O4 + YMn2O5,
 Y2O3 + 1.275 Mn2O3 -> 0.975 Y2Mn2O7 + 0.05 YMn12,
 Y2O3 + 2.25 Mn2O3 -> 0.05 YMn12 + 1.95 YMn2O5,
 MnO + 0.6667 Y2O3 -> YMnO3 + 0.3333 Y,
 Mn + Y2O3 -> YMnO3 + Y,
 Y2O3 + 2 Mn2O3 -> Y2Mn2O7 + 2 MnO,
 2 Mn5O8 + 3 Y2O3 -> Y + 5 YMn2O5,
 Y2O3 + 1.333 Mn2O3 -> Y2Mn2O7 + 0.6667 Mn,
 2 Mn5O8 + Y2O3 -> Y2Mn2O7 + 4 Mn2O3,
 Mn3O4 + 1.069 Y2O3 -> 1.029 Y2Mn2O7 + 0.07843 YMn12,
 Mn + 0.5 Y2O3 -> Y + 0.5 Mn2O3,
 Mn5O8 + 2.069 Y2O3 -> 2.029 Y2Mn2O7 + 0.07843 YMn12,
 Y2O3 + 4 Mn2O3 -> Y2Mn2O7 + 2 Mn3O4,
 2 Mn + 2.333 Y2O3 -> Y2Mn2O7 + 2.667 Y,
 Y2O3 + 2.55 YMn2O5 -> 2.25 Y2Mn2O7 + 0.05 YMn12,
 Mn5O8 + 1.167 Y2O3 -> 0.03333 YMn12 + 2.3 YMn2O5,
 Mn3O4 + 0.01932 Y2O3 -> 0

This same approach can be used for the target phase(s) as well.

With `exclusive_targets=False`:

In [16]:
be_target = BasicEnumerator(targets=["YMnO3"])
ymno3_rxns = be_target.enumerate(filtered_entries)

BasicEnumerator:   0%|          | 0/4 [00:00<?, ?it/s]

In [17]:
ymno3_rxns

[0.02564 YMn12 + 1.359 Mn2O3 -> Mn3O4 + 0.02564 YMnO3,
 0.6667 Y2Mn2O7 + 0.3333 Mn -> YMnO3 + 0.3333 YMn2O5,
 MnO + YMn2O5 -> YMnO3 + Mn2O3,
 4.667 Mn5O8 + YMn12 -> YMnO3 + 34.33 MnO,
 0.5 Y2Mn2O7 + Mn3O4 -> YMnO3 + 1.5 Mn2O3,
 Y2Mn2O7 + 0.625 Mn -> 2 YMnO3 + 0.125 Mn5O8,
 0.375 Mn5O8 + Y -> YMnO3 + 0.875 Mn,
 Y + 2 Mn2O3 -> YMnO3 + 3 MnO,
 MnO + 0.6667 Y2O3 -> YMnO3 + 0.3333 Y,
 Mn + Y2O3 -> YMnO3 + Y,
 2 Mn3O4 + Y -> YMnO3 + 5 MnO,
 YMn12 + 1.5 O2 -> YMnO3 + 11 Mn,
 0.4882 Y2Mn2O7 + 0.0237 YMn12 -> YMnO3 + 0.05213 Mn5O8,
 0.5 Y2Mn2O7 + 0.3333 Mn -> YMnO3 + 0.1667 Mn2O3,
 Mn3O4 + YMn2O5 -> YMnO3 + 2 Mn2O3,
 Mn5O8 + 2.861 Y -> 2.667 YMnO3 + 0.1944 YMn12,
 Mn3O4 + 2 YMn2O5 -> 2 YMnO3 + Mn5O8,
 0.5 Y2Mn2O7 + 0.5 Mn -> YMnO3 + 0.5 MnO,
 0.75 Mn3O4 + YMn12 -> YMnO3 + 13.25 Mn,
 0.025 YMn12 + 0.975 YMn2O5 -> YMnO3 + 0.625 Mn2O3,
 14 Mn3O4 + YMn12 -> YMnO3 + 53 MnO,
 51.5 MnO2 + YMn12 -> YMnO3 + 12.5 Mn5O8,
 YMn12 + 14 Mn2O3 -> YMnO3 + 39 MnO,
 0.4 Y + 0.6 YMn2O5 -> YMnO3 + 0.2 Mn,
 0.5 Y2Mn

With `exclusive_targets=True`:

In [18]:
be_target = BasicEnumerator(targets=["YMnO3"], exclusive_targets=True)
ymno3_rxns = be_target.enumerate(filtered_entries)

BasicEnumerator:   0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
ymno3_rxns

[Y2O3 + Mn2O3 -> 2 YMnO3]

And finally with multiple targets (e.g., YMnO3 and O2):

In [20]:
be_targets = BasicEnumerator(targets=["YMnO3", "O2"], exclusive_targets=True)
ymno3_rxns_o2 = be_targets.enumerate(filtered_entries)

BasicEnumerator:   0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
ymno3_rxns_o2

[Y2O3 + Mn2O3 -> 2 YMnO3,
 0.25 Y2O3 + 0.5 YMn2O5 -> YMnO3 + 0.125 O2,
 0.5 Y2Mn2O7 -> YMnO3 + 0.25 O2,
 0.2 Mn5O8 + 0.5 Y2O3 -> YMnO3 + 0.05 O2,
 MnO2 + 0.5 Y2O3 -> YMnO3 + 0.25 O2]

#### Open entries

In the previous cell, we showed that it was possible to specify YMnO3 as a target, along with O2. However, because O2 is a gas, it is often desirable to include it as an **open entry** in addition to the 1-2 possible precursors/targets. For example, we may want to specify a reaction that follows:

$$ A + B ~ (+~O_2) \rightarrow C + D ~ (+~O_2) $$

To do this, we use the `BasicOpenEnumerator` class, which is an extension to the previous basic enumerator. All of the lessons learned above also apply to this class, although now a list of open entry formulas must be specified.

In [22]:
be_target_open = BasicOpenEnumerator(["O2"],targets=["YMnO3"])
ymno3_rxns_open = be_target.enumerate(filtered_entries)

BasicEnumerator:   0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
ymno3_rxns_open

[Y2O3 + Mn2O3 -> 2 YMnO3]

You will now see that the reactions from before are included in this new list, along with many other open-O2 reactions -- some which even have 3 reactants or 3 products. In other words, the open entry/entries do not count towards the specified $n$ of the reactions.

It is even possible to specify multiple open entries, allowing for even more complex reactions (such as ones with 4+ reactants or products). For example, specifiyng Y2O3 and O2 as both being open:

In [24]:
be_target_open2 = BasicOpenEnumerator(["Y2O3", "O2"],targets=["YMnO3"])
ymno3_rxns_open2 = be_target_open2.enumerate(filtered_entries)

BasicOpenEnumerator:   0%|          | 0/1 [00:00<?, ?it/s]

In [25]:
ymno3_rxns_open2

[Y2Mn2O7 + 4 Mn3O4 + O2 + Y2O3 -> 4 YMnO3 + 2 Mn5O8,
 0.4045 MnO2 + 0.3596 YMn2O5 -> 0.04494 YMnO3 + 1.079 Mn + O2 + 0.1573 Y2O3,
 3.5 Mn5O8 + 3 Y -> 5.5 Mn3O4 + YMnO3 + Y2O3,
 0.4162 Y2Mn2O7 + 0.1675 YMn2O5 -> 0.05584 Mn3O4 + YMnO3 + 0.264 O2,
 3 MnO + 3 YMn2O5 -> YMnO3 + Y2O3 + 4 Mn2O3,
 Mn3O4 + YMn12 + 10 O2 -> YMnO3 + 7 Mn2O3,
 4 MnO2 + 1.667 Y -> Mn3O4 + YMnO3 + 0.3333 Y2O3,
 0.2857 Y + YMn2O5 -> 1.286 YMnO3 + 0.7143 Mn + 0.5714 O2,
 YMn12 + 13 O2 + Mn2O3 -> YMnO3 + 13 MnO2,
 1.812 MnO + Y2O3 + 0.5454 YMn2O5 -> 2.513 YMnO3 + 0.03247 YMn12,
 Y2Mn2O7 -> YMnO3 + 0.25 O2 + MnO2 + 0.5 Y2O3,
 0.2059 Mn5O8 + 0.1765 YMn2O5 -> 0.1765 YMnO3 + 1.206 Mn + O2,
 1.594 Mn3O4 + O2 + Y2O3 + 0.6604 Y -> 2.66 YMnO3 + 0.4245 Mn5O8,
 0.625 Y2Mn2O7 + 0.25 Mn3O4 -> YMnO3 + MnO2 + 0.125 Y2O3,
 15.17 Y + 6.5 YMn2O5 -> YMnO3 + YMn12 + 9.833 Y2O3,
 15 Mn3O4 + YMn12 + Y2O3 -> 3 YMnO3 + 54 MnO,
 0.02766 Y2O3 + MnO2 + 0.6645 Y -> 0.6943 YMnO3 + 0.02547 YMn12,
 2.5 Y2Mn2O7 + 0.5 Mn2O3 -> YMnO3 + Mn5O8 + O2 + 2 

This may be useful for systems where more than 1 gaseous, liquid, or molten phases are available to the reacting system, such as $O_2$ and a molten salt (e.g. $LiCl$)

### Minimize Enumerators

The "minimize" enumerators produce reactions via a thermodynamic approach, rather than a purely combinatorial one. This means that reactions are produced from a phase diagram approach, where a new convex hull is drawn connecting two compositions within a closed (Gibbs) or open (Grand Potential) system. See the `InterfacialReactivity` class within the _pymatgen_ package for more information.

It is **important to note** that reactions produced with the minimize enumerators may overlap some with the basic enumerators, but the minimize enumerators have the restriction that all original reactions produced must have a negative reaction energy and result in a set of product phases which are stable with respect to each other (i.e. they share a facet of the phase diagram).

Even though these enumerators also yield the reverse reactions to those described above, it is impossible for them to identify a reaction where both sides of the reaction contain phases that are unstable with respect to each other. In other words, every reaction proudced has either reactants that share a phase diagram facet or products that share a phase diagram facet).

The `MinimizeGibbsEnumerator` contains similar arguments as the basic enumerators. See the docstrings for updated information:
- **precursors**: Optional formulas of precursors.
- **targets**: Optional formulas of targets; only reactions which make these targets
    will be enumerated.
- **calculators**: Optional list of Calculator object names; see calculators
    module for options (e.g., ["ChempotDistanceCalculator"])
- **exclusive_precursors**: Whether to consider only reactions that have
    reactants which are a subset of the provided list of precursors.
    Defaults to True.
- **exclusive_targets**: Whether to consider only reactions that make the
    provided target directly (i.e. with no byproducts). Defualts to False.

In [26]:
mge = MinimizeGibbsEnumerator()

The default arguments, as before, help produce all reactions in a set of entries, given minimal constraints.

In [27]:
rxns = mge.enumerate(filtered_entries)

MinimizeGibbsEnumerator:   0%|          | 0/4 [00:00<?, ?it/s]

In [28]:
pprint(rxns[0:10])

[YMn12 + 53 Mn2O3 -> YMnO3 + 39 Mn3O4,
 0.2571 Mn3O4 + 0.01905 YMn12 -> MnO + 0.009524 Y2O3,
 0.2 Mn5O8 + 1.067 Y -> Mn + 0.5333 Y2O3,
 0.3333 Mn + 0.6667 Y2Mn2O7 -> YMnO3 + 0.3333 YMn2O5,
 0.3333 Y + YMnO3 -> MnO + 0.6667 Y2O3,
 0.07843 YMn12 + 0.02941 Y2Mn2O7 -> Mn + 0.06863 Y2O3,
 2.5 Mn2O3 + 0.5 Y2O3 -> YMn2O5 + Mn3O4,
 4.667 Mn5O8 + YMn12 -> YMnO3 + 34.33 MnO,
 0.375 Mn5O8 + 0.125 Mn -> Mn2O3,
 0.5 Mn + 0.5 MnO2 -> MnO]


And as before, we can specify various combinations of precursors and targets, as well as whether or not they should be "exclusive".

In [29]:
mge_precursors = MinimizeGibbsEnumerator(precursors=["Y2O3"], exclusive_precursors=False)
rxns = mge_precursors.enumerate(filtered_entries)
rxns

MinimizeGibbsEnumerator:   0%|          | 0/4 [00:00<?, ?it/s]

[0.3333 Mn5O8 + 0.6667 Y2O3 -> YMnO3 + 0.3333 YMn2O5,
 0.625 Mn5O8 + 0.5 Y2O3 -> YMn2O5 + 0.375 Mn3O4,
 2 MnO2 + Y2O3 -> Y2Mn2O7,
 0.5 Mn2O3 + 0.5 Y2O3 -> YMnO3,
 0.5 Mn3O4 + 0.5 Y2O3 -> YMnO3 + 0.5 MnO,
 0.6667 Mn5O8 + 0.3333 Y2O3 -> Mn2O3 + 0.6667 YMn2O5,
 2.5 Mn2O3 + 0.5 Y2O3 -> YMn2O5 + Mn3O4,
 2 MnO2 + 0.5 Y2O3 -> YMn2O5 + 0.25 O2]

In [30]:
mge_precursors = MinimizeGibbsEnumerator(precursors=["Y2O3", "Mn2O3"])
rxns = mge_precursors.enumerate(filtered_entries)
rxns

MinimizeGibbsEnumerator:   0%|          | 0/1 [00:00<?, ?it/s]

[2.5 Mn2O3 + 0.5 Y2O3 -> YMn2O5 + Mn3O4, 0.5 Mn2O3 + 0.5 Y2O3 -> YMnO3]

In [31]:
mge_targets = MinimizeGibbsEnumerator(targets=["YMnO3"], exclusive_targets=True)
rxns = mge_targets.enumerate(filtered_entries)
rxns

MinimizeGibbsEnumerator:   0%|          | 0/1 [00:00<?, ?it/s]

[0.5 Mn2O3 + 0.5 Y2O3 -> YMnO3]

In [32]:
mge_targets = MinimizeGibbsEnumerator(targets=["YMnO3"], exclusive_targets=False)
rxns = mge_targets.enumerate(filtered_entries)
rxns

MinimizeGibbsEnumerator:   0%|          | 0/4 [00:00<?, ?it/s]

[YMn12 + 53 Mn2O3 -> YMnO3 + 39 Mn3O4,
 YMn12 + 14 Mn2O3 -> YMnO3 + 39 MnO,
 0.5 Mn2O3 + 0.5 Y2O3 -> YMnO3,
 0.3333 Y + 0.5 Y2Mn2O7 -> YMnO3 + 0.1667 Y2O3,
 14 MnO2 + YMn12 -> YMnO3 + 25 MnO,
 0.06667 YMn12 + 0.9333 YMn2O5 -> YMnO3 + 1.667 MnO,
 0.2857 Y + 0.7143 YMn2O5 -> YMnO3 + 0.1429 Mn3O4,
 0.5 Mn + YMn2O5 -> YMnO3 + 0.5 Mn3O4,
 YMn12 + 8.833 O2 -> YMnO3 + 3.667 Mn3O4,
 0.3333 Mn + 0.6667 Y2Mn2O7 -> YMnO3 + 0.3333 YMn2O5,
 Mn3O4 + 2 Y2Mn2O7 -> YMnO3 + 3 YMn2O5,
 Mn + YMn2O5 -> YMnO3 + 2 MnO,
 YMn12 + 7 O2 -> YMnO3 + 11 MnO,
 4.667 Mn5O8 + YMn12 -> YMnO3 + 34.33 MnO,
 2 MnO2 + Y -> YMnO3 + MnO,
 0.03448 YMn12 + 0.4828 Y2Mn2O7 -> YMnO3 + 0.3793 MnO,
 5 Mn2O3 + Y -> YMnO3 + 3 Mn3O4,
 2 Mn2O3 + Y -> YMnO3 + 3 MnO,
 13.25 Mn5O8 + YMn12 -> YMnO3 + 25.75 Mn3O4,
 1.5 MnO + 0.5 Y2Mn2O7 -> YMnO3 + 0.5 Mn3O4,
 MnO + Y2Mn2O7 -> YMnO3 + YMn2O5,
 0.025 YMn12 + 0.625 Y2Mn2O7 -> YMnO3 + 0.275 YMn2O5,
 2 Mn3O4 + Y -> YMnO3 + 5 MnO,
 0.6667 Mn5O8 + Y -> YMnO3 + 2.333 MnO,
 0.3333 Y + 0.6667 YMn2O5 

#### Open entries

And once again, as before, we can do all the same analysis with open entries. This time, the grand potential is used as the thermodynamic free energy which is minimized:

$$ \Phi = G - \mu_iN_i $$

Where $i$ is the open species with chemical potential $\mu_i$ with a molar amount $N_i$.

In [33]:
mgpe = MinimizeGrandPotentialEnumerator(open_elem=Element("O"), mu=0)
open_rxns = mgpe.enumerate(filtered_entries)
open_rxns

MinimizeGrandPotentialEnumerator:   0%|          | 0/3 [00:00<?, ?it/s]

[YMn12 + 12.5 O2 -> YMn2O5 + 10 MnO2 (mu_O=0),
 2 YMnO3 + 0.5 O2 -> Y2Mn2O7 (mu_O=0),
 2 Y + 1.5 O2 -> Y2O3 (mu_O=0),
 Y + YMn2O5 + O2 -> Y2Mn2O7 (mu_O=0),
 0.09091 YMn12 + 0.9091 YMnO3 + 1.136 O2 -> YMn2O5 (mu_O=0),
 0.5 Mn2O3 + 0.25 O2 -> MnO2 (mu_O=0),
 2 MnO2 + 2 Y + 1.5 O2 -> Y2Mn2O7 (mu_O=0),
 Mn2O3 + Y2O3 + 0.5 O2 -> Y2Mn2O7 (mu_O=0),
 MnO + YMnO3 + 0.5 O2 -> YMn2O5 (mu_O=0),
 Mn2O3 + Y + O2 -> YMn2O5 (mu_O=0),
 Mn + YMnO3 + O2 -> YMn2O5 (mu_O=0),
 0.6667 Mn3O4 + 0.5 Y2O3 + 0.4167 O2 -> YMn2O5 (mu_O=0),
 2 MnO + 0.5 Y2O3 + 0.75 O2 -> YMn2O5 (mu_O=0),
 2 MnO + Y + 1.5 O2 -> YMn2O5 (mu_O=0),
 0.1667 YMn12 + 0.9167 Y2O3 + 2.125 O2 -> Y2Mn2O7 (mu_O=0),
 0.4 Mn5O8 + 0.5 Y2O3 + 0.15 O2 -> YMn2O5 (mu_O=0),
 2 Mn + Y2O3 + 2 O2 -> Y2Mn2O7 (mu_O=0),
 0.6667 Mn3O4 + 2 Y + 2.167 O2 -> Y2Mn2O7 (mu_O=0),
 0.6667 Mn3O4 + Y2O3 + 0.6667 O2 -> Y2Mn2O7 (mu_O=0),
 0.3333 Mn3O4 + YMnO3 + 0.3333 O2 -> YMn2O5 (mu_O=0),
 0.4 Mn5O8 + Y2O3 + 0.4 O2 -> Y2Mn2O7 (mu_O=0),
 2 MnO + Y2O3 + O2 -> Y2Mn2O7 (mu_O

Note that the reaction objects returned are now of a different type: `OpenComputedReaction`. This class allows for easy specification of reactions where one of the elements is assigned a chemical potential.

In [34]:
r = open_rxns[0]
print(r.__class__.__name__)

OpenComputedReaction


We can also, as before, customize which precursors and targets are specified:

In [35]:
mgpe_precursors = MinimizeGrandPotentialEnumerator(open_elem=Element("O"), mu=0,
                                                   precursors=["Y2O3"], exclusive_precursors=False)
open_rxns_precursors = mgpe_precursors.enumerate(filtered_entries)
open_rxns_precursors

MinimizeGrandPotentialEnumerator:   0%|          | 0/3 [00:00<?, ?it/s]

[0.1667 YMn12 + 0.9167 Y2O3 + 2.125 O2 -> Y2Mn2O7 (mu_O=0),
 0.6667 Mn3O4 + 0.5 Y2O3 + 0.4167 O2 -> YMn2O5 (mu_O=0),
 2 Mn + 0.5 Y2O3 + 1.75 O2 -> YMn2O5 (mu_O=0),
 0.4 Mn5O8 + 0.5 Y2O3 + 0.15 O2 -> YMn2O5 (mu_O=0),
 2 Mn + Y2O3 + 2 O2 -> Y2Mn2O7 (mu_O=0),
 Mn2O3 + 0.5 Y2O3 + 0.25 O2 -> YMn2O5 (mu_O=0),
 0.5 Y2O3 + YMn2O5 + 0.25 O2 -> Y2Mn2O7 (mu_O=0),
 Mn2O3 + Y2O3 + 0.5 O2 -> Y2Mn2O7 (mu_O=0),
 0.6667 Mn3O4 + Y2O3 + 0.6667 O2 -> Y2Mn2O7 (mu_O=0),
 0.4 Mn5O8 + Y2O3 + 0.4 O2 -> Y2Mn2O7 (mu_O=0),
 0.1667 YMn12 + 0.4167 Y2O3 + 1.875 O2 -> YMn2O5 (mu_O=0),
 2 MnO + Y2O3 + O2 -> Y2Mn2O7 (mu_O=0),
 2 MnO + 0.5 Y2O3 + 0.75 O2 -> YMn2O5 (mu_O=0),
 8 MnO2 + 2 Y2O3 -> O2 + 4 YMn2O5 (mu_O=0)]

In [36]:
mgpe_precursors = MinimizeGrandPotentialEnumerator(open_elem=Element("O"), mu=0,
                                                   targets=["Y2Mn2O7"])
open_rxns_targets = mgpe_precursors.enumerate(filtered_entries)
open_rxns_targets

MinimizeGrandPotentialEnumerator:   0%|          | 0/3 [00:00<?, ?it/s]

[0.1667 YMn12 + 0.9167 Y2O3 + 2.125 O2 -> Y2Mn2O7 (mu_O=0),
 2 Mn + Y2O3 + 2 O2 -> Y2Mn2O7 (mu_O=0),
 2 YMnO3 + 0.5 O2 -> Y2Mn2O7 (mu_O=0),
 0.6667 Mn3O4 + 2 Y + 2.167 O2 -> Y2Mn2O7 (mu_O=0),
 0.5 Y2O3 + YMn2O5 + 0.25 O2 -> Y2Mn2O7 (mu_O=0),
 Mn2O3 + 2 Y + 2 O2 -> Y2Mn2O7 (mu_O=0),
 0.4 Mn5O8 + 2 Y + 1.9 O2 -> Y2Mn2O7 (mu_O=0),
 Y + YMn2O5 + O2 -> Y2Mn2O7 (mu_O=0),
 2 MnO2 + 2 Y + 1.5 O2 -> Y2Mn2O7 (mu_O=0),
 Mn2O3 + Y2O3 + 0.5 O2 -> Y2Mn2O7 (mu_O=0),
 2 MnO + 2 Y + 2.5 O2 -> Y2Mn2O7 (mu_O=0),
 0.6667 Mn3O4 + Y2O3 + 0.6667 O2 -> Y2Mn2O7 (mu_O=0),
 0.4 Mn5O8 + Y2O3 + 0.4 O2 -> Y2Mn2O7 (mu_O=0),
 2 MnO + Y2O3 + O2 -> Y2Mn2O7 (mu_O=0)]

Note that setting the chemical potential to a value outside of the range of stability of the target causes the enumerator to yield no reactions:

In [37]:
mgpe_precursors = MinimizeGrandPotentialEnumerator(open_elem=Element("O"), mu=-3,
                                                   targets=["Y2Mn2O7"])
open_rxns_targets = mgpe_precursors.enumerate(filtered_entries)
open_rxns_targets

MinimizeGrandPotentialEnumerator:   0%|          | 0/3 [00:00<?, ?it/s]

[]

## Running enumerators with the Fireworks package

A convenient way to run one or more enumerators is to use the `EnumeratorFW`, a wrapper for running enumerators that is written as a `Firework` within the _fireworks_ package. This allows for easy workflow management, such as queue submissions, I/O serialization, tracking of calculations, etc.

The `EnumeratorFW` has support for acquiring entries either from a custom entry database (MongoDB), or from the Materials Project using `MPRester` API. To set up the firework, we simply provide a list of enumerators, and any of the optional parameters. If a chemical system is provided, entires for that system will be acquired before running the enumerator(s).

In [38]:
from rxn_network.fireworks import EnumeratorFW
from fireworks import Workflow, LaunchPad

In [42]:
be = BasicEnumerator()
fw = EnumeratorFW([be], chemsys="Y-Mn-O", entry_set_params={"e_above_hull":0})

The firework is simply a dictionary with a list of FireTasks where 1) entries are acquired, 2) enumerators are run, and 3) reactions are stored in a specified database

We then place the firework by itself into a `Workflow` so that it can be added to our fireworks `LaunchPad` (see the fireworks documentation for more info on setting this up):

In [56]:
wf = Workflow([fw], name="Y-Mn-O Enumerator")

In [57]:
lpad = LaunchPad.auto_load()
lpad.add_wf(wf)

2021-12-14 15:55:12,046 INFO Added a workflow. id_map: {3846: 3847}


{3846: 3847}

The workflow can now either be launched through a queue submission (i.e. for supercomputer use) or simply on your local machine, by running `rlaunch singleshot` in your terminal as below. **Note**: this will launch in whatever folder this notebook is in!

In [58]:
!rlaunch singleshot

2021-12-14 15:55:13,153 INFO Hostname/IP lookup (this will take a few seconds)
2021-12-14 15:55:13,154 INFO Launching Rocket
2021-12-14 15:55:16,196 INFO RUNNING fw_id: 3847 in directory: /Users/mcdermott/PycharmProjects/reaction-network/notebooks
2021-12-14 15:55:16,233 INFO Task started: {{rxn_network.firetasks.build_inputs.EntriesFromDb}}.
2021-12-14 15:55:17,552 INFO Task completed: {{rxn_network.firetasks.build_inputs.EntriesFromDb}} 
2021-12-14 15:55:17,567 INFO Task started: {{rxn_network.firetasks.run_calc.RunEnumerators}}.
O-Y: 100%|████████████████████████████████████████| 4/4 [00:01<00:00,  2.84it/s]
2021-12-14 15:55:19,219 INFO Task completed: {{rxn_network.firetasks.run_calc.RunEnumerators}} 
2021-12-14 15:55:19,273 INFO Task started: {{rxn_network.firetasks.parse_outputs.ReactionsToDb}}.
2021-12-14 15:55:19,335 INFO rxn_network.utils.database Updating /Users/mcdermott/PycharmProjects/reaction-network/notebooks with taskid = 82
2021-12-14 15:55:19,389 INFO Task completed: 

If any errors are encountered, please double-check that your Fireworks settings are configured properly. If the error persists, please raise an Issue here: https://github.com/GENESIS-EFRC/reaction-network/issues